# Prepare Data

As we analysed in Comp_EDA.ipynb, build a Machine Learning model based on the types and Country Group.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math

In [2]:
# Matplotlib Config
sns.set()
plt.rcParams["figure.figsize"] = (14,8)
plt.style.use('fivethirtyeight')

In [3]:
df_result = pd.read_csv('../data/2019/survey_results_public.csv')
print(df_result.shape)
df_result.head()

(88883, 85)


,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [4]:
# Load Country Group Dictionary
with open('../data/2019/country_group_dict.json', 'r') as f:
    country_group_dict = json.load(f)

country_group_dict

{'United States': 'US',
 'Switzerland': 'Group A',
 'Israel': 'Group A',
 'Denmark': 'Group A',
 'Norway': 'Group A',
 'Australia': 'Group A',
 'Ireland': 'Group A',
 'Canada': 'Group A',
 'United Kingdom': 'Group A',
 'New Zealand': 'Group B',
 'Singapore': 'Group B',
 'Germany': 'Group B',
 'Netherlands': 'Group B',
 'Hong Kong (S.A.R.)': 'Group B',
 'Sweden': 'Group B',
 'Japan': 'Group B',
 'Finland': 'Group B',
 'United Arab Emirates': 'Group B',
 'Belgium': 'Group B',
 'Austria': 'Group B',
 'France': 'Group C',
 'South Korea': 'Group C',
 'South Africa': 'Group C',
 'Thailand': 'Group C',
 'Spain': 'Group C',
 'Estonia': 'Group C',
 'Latvia': 'Group C',
 'Uruguay': 'Group C',
 'Slovenia': 'Group C',
 'Italy': 'Group C',
 'Czech Republic': 'Group C',
 'Lithuania': 'Group C',
 'Poland': 'Group C',
 'Romania': 'Group C',
 'Slovakia': 'Group C',
 'Chile': 'Group C',
 'Taiwan': 'Group C',
 'Portugal': 'Group C',
 'Bulgaria': 'Group D',
 'China': 'Group D',
 'Hungary': 'Group D',
 'Cr

In [5]:
# Create Country List
country_list = list(country_group_dict.keys())
country_group_list = set(country_group_dict.values())

In [6]:
country_group_list

{'Group A', 'Group B', 'Group C', 'Group D', 'US'}

In [7]:
# Clean data as analysed in EDA
def clean_data(df):
    """Drop records, fill Age and Years"""
    df_new = df.copy()
    # Drop rows with missing values in Country or ConvertedComp
    df_new.dropna(subset=['Country', 'ConvertedComp'], axis=0, how='any', inplace=True)
    # Drop rows with missing values in YearsCode and YearsCodePro
    df_new.dropna(subset=['YearsCode', 'YearsCodePro'], axis=0, how='all', inplace=True)
    # Drop rows for Full-time Students
    df_new.drop(df_new[df_new['Student']=='Yes, full-time'].index, inplace=True)

    df_new['YearsCodePro'].replace('Less than 1 year', '0.5', inplace=True)
    df_new['YearsCodePro'].replace('More than 50 years', '60', inplace=True)
    df_new['YearsCodePro'] = df_new['YearsCodePro'].astype(float)

    df_new['YearsCode'].replace('Less than 1 year', '0.5', inplace=True)
    df_new['YearsCode'].replace('More than 50 years', '60', inplace=True)
    df_new['YearsCode'] = df_new['YearsCode'].astype(float)
    
    age_mean = df[['Country', 'Age']].dropna().groupby(by='Country').mean().round(decimals=1)
    age_mean.reset_index(inplace=True)
    
    for idx, row in df_new.iterrows():
        # Fill the mean of the Country if Age is null
        if math.isnan(row['Age']):
            mean = age_mean.loc[age_mean['Country'] == row['Country']]['Age'].values
            if mean.size > 0:
                df_new.loc[df_new.index == idx, 'Age'] = mean[0]
        # Fill YearsCode or YearsCodePro if either is missing
        if math.isnan(row['YearsCode']):
            df_new.loc[df_new.index == idx, 'YearsCode'] == row['YearsCodePro']
        if math.isnan(row['YearsCodePro']):
            df_new.loc[df_new.index == idx, 'YearsCodePro'] == row['YearsCode']
    
    # Drop rows with missing values in YearsCode and YearsCodePro
    df_new.dropna(subset=['YearsCode', 'YearsCodePro'], axis=0, how='any', inplace=True)
    
    return df_new

In [8]:
# Clean data
df_cleaned = clean_data(df_result)
df_cleaned['Age'].isnull().sum()

0

In [9]:
def trim_data(df, max_comp, max_comp_us, min_comp, min_age, countries):
    """Return a cleaned DataFrame for the given DataFrame using the given max/min for Comps and Age"""
    df_new = df
    # Drop rows out of the given range
    df_new = df_new.loc[(df_new['ConvertedComp'] >= min_comp) &
                        (((df_new['Country'] == "United States") &
                          (df_new['ConvertedComp'] <= max_comp_us)) |
                         ((df_new['Country'] != "United States") &
                          (df_new['ConvertedComp'] <= max_comp))) &
                        (df_new['Age'] > min_age)
                       ].sort_values(by='ConvertedComp', ascending=False)
        
    # Filter out countries
    if len(countries) > 0:
        df_new = df_new.loc[df_new['Country'].isin(countries)]
    
    return df_new

In [10]:
# Trim by ConvertedComp and Age threshold and filter countries
df_trimmed = trim_data(df_cleaned, 250000, 500000, 1000, 10, country_list)

# Add Country Group
df_trimmed['CountryGroup'] = df_trimmed['Country'].apply(lambda x: country_group_dict[x])

# Rename ConvertedComp to Salary
df_trimmed.rename(columns={'ConvertedComp': 'Salary'}, inplace=True)

# Reset index
df_trimmed.reset_index(inplace=True)

df_trimmed.shape

(43836, 87)

## Prepare X and Y

In [11]:
# Prepare X and Y
X = df_trimmed[['Country', 'CountryGroup', 'Age', 'YearsCode', 'YearsCodePro', 'DevType', 'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'WebFrameWorkedWith', 'MiscTechWorkedWith']]
y = df_trimmed['Salary']

In [12]:
X.head()

,Country,CountryGroup,Age,YearsCode,YearsCodePro,DevType,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,WebFrameWorkedWith,MiscTechWorkedWith
0,United States,US,32.8,38.0,10.0,Engineering manager,Assembly;Bash/Shell/PowerShell;C++;C#;Dart;Go;...,NaN,Docker;iOS;Linux;MacOS;Other(s):,NaN,Flutter;.NET
1,United States,US,42.0,23.0,18.0,"Database administrator;Developer, back-end;Dev...",C++;Java;JavaScript;Kotlin;Python;TypeScript,MySQL,Android;AWS;Linux;Slack,Django;Vue.js,Apache Spark
2,United States,US,41.0,33.0,21.0,"Developer, back-end;Developer, front-end;Devel...",HTML/CSS;JavaScript;TypeScript,NaN,Linux;MacOS;Windows,Angular/Angular.js,NaN
3,United States,US,38.0,22.0,17.0,"Developer, embedded applications or devices",Assembly;Bash/Shell/PowerShell;C;C++;Clojure,NaN,Android;iOS;MacOS,NaN,NaN
4,United States,US,44.0,30.0,21.0,"Developer, game or graphics",C;C++;Objective-C;Swift,NaN,MacOS,NaN,TensorFlow


In [13]:
y.head()

0    500000.0
1    500000.0
2    500000.0
3    500000.0
4    500000.0
Name: Salary, dtype: float64

In [14]:
def onehot_encode(df, columns):
    """Returns a DataFrame after onehot encoding on given columns"""
    df_new = df.copy()
    for column in columns:
        # Split multiple labels into a list
        df_new.fillna(value='', inplace=True)
        df_new.loc[:, column] = df_new[column].str.split(pat=';')
        # pd.get_dummies(X, columns=['DevType']) # this does not work as each value is a list
        # X.drop('DevType', 1).join(X.DevType.str.join('|').str.get_dummies()) # cannot add prefix
        mask = df_new[column].notnull()
        prefix_string = column[0:2]
        df_new = df_new.drop(column, 1).join(
            pd.get_dummies(
                pd.DataFrame(df_new.loc[mask, column].tolist()).stack(), prefix=column
            ).astype(int).sum(level=0)
        )
    return df_new

In [15]:
# Drop Country and One-hot encode
X = pd.get_dummies(X, columns=['CountryGroup']).drop(columns=['Country'])
column_list = ['DevType', 'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'WebFrameWorkedWith', 'MiscTechWorkedWith']
X = onehot_encode(X, column_list)

X.head()

,Age,YearsCode,YearsCodePro,CountryGroup_Group A,CountryGroup_Group B,CountryGroup_Group C,CountryGroup_Group D,CountryGroup_US,DevType_,DevType_Academic researcher,...,MiscTechWorkedWith_Node.js,MiscTechWorkedWith_Other(s):,MiscTechWorkedWith_Pandas,MiscTechWorkedWith_Puppet,MiscTechWorkedWith_React Native,MiscTechWorkedWith_TensorFlow,MiscTechWorkedWith_Torch/PyTorch,MiscTechWorkedWith_Unity 3D,MiscTechWorkedWith_Unreal Engine,MiscTechWorkedWith_Xamarin
0,32.8,38.0,10.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42.0,23.0,18.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,41.0,33.0,21.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,38.0,22.0,17.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44.0,30.0,21.0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
X.isnull().sum().tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# Create Model and Train

In [17]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import tree
from sklearn import svm

In [18]:
def estimate_salary(df, country_group_dict, country_list, country_group_list):
    """All in one function to clean data, create model and estimate"""
    df_new = df.copy()
    # Clean Data
    print("Cleaning the data...")
    df_new = clean_data(df_new)
    print("Cleaning the data... done.")
    
    # Trim by ConvertedComp and Age threshold and filter countries
    print("Trimming the data...")
    df_new = trim_data(df_new, 250000, 500000, 1000, 10, country_list)
    print("Trimming the data... done.")

    # Add Country Group
    df_new['CountryGroup'] = df_new['Country'].apply(lambda x: country_group_dict[x])

    # Rename ConvertedComp to Salary
    df_new.rename(columns={'ConvertedComp': 'Salary'}, inplace=True)

    # Reset index
    df_new.reset_index(inplace=True)

    # Prepare X and Y
    X = df_new[['Country', 'CountryGroup', 'Age', 'YearsCode', 'YearsCodePro', 'DevType', 'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'WebFrameWorkedWith', 'MiscTechWorkedWith']]
    y = df_new['Salary']
    
    #X = X.drop(columns=['DevType', 'LanguageWorkedWith'])
    
    # One-hot encode
    print("One-hot encoding...")
    column_list = ['DevType', 'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'WebFrameWorkedWith', 'MiscTechWorkedWith']
    X = onehot_encode(X, column_list)
    print("One-hot encoding... done.")
    
    # Linear Regression model
    lm_model = LinearRegression(normalize=True)
    lg_model = LogisticRegression(random_state=42)
    tree_model = tree.DecisionTreeRegressor()
    svm_model = svm.SVR()
    
    
    model_selected = lm_model
    
    def train_model(model, X, y):
        """Perform training and return r2 score on test and training set and the model"""
        # Split into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)
        
        # Fit
        model.fit(X_train, y_train)
        
        # Predict and score the model
        y_pred = model.predict(X_test)
        y_pred_train = model.predict(X_train)
        test_score = r2_score(y_test, y_pred)
        train_score = r2_score(y_train, y_pred_train)
        
        return test_score, train_score, model    
    
    test_score_list, train_score_list, model_list, name_list = [], [], [], []
    
    # Training for all data
    print("Trainning on: Without Country")
    # Drop Country and One-hot encode by CountryGroup
    X_without = X.drop(columns=['Country', 'CountryGroup'])
    test_score, train_score, model = train_model(model_selected, X_without, y)
    test_score_list.append(test_score)
    train_score_list.append(train_score)
    model_list.append(model)
    name_list.append('Without Country')
    print("Training... done.")
    
    # Training for all data
    print("Trainning on: With Country")
    # Drop Country and One-hot encode by CountryGroup
    X_all = pd.get_dummies(X, columns=['CountryGroup']).drop(columns=['Country'])
    test_score, train_score, model = train_model(model_selected, X_all, y)
    test_score_list.append(test_score)
    train_score_list.append(train_score)
    model_list.append(model)
    name_list.append('With Country')
    print("Training... done.")
    
    # Training per Country Group
    df_group = X
    df_group['Salary'] = y
    for group in country_group_list:
        print("Trainning on: ", group)
        print("Filtering data...")    
        df_group_filtered = df_group.loc[df_group['CountryGroup']==group].drop(columns=['CountryGroup', 'Country'])
        X_group = df_group_filtered.drop(columns=['Salary'])
        y_group = df_group_filtered['Salary']
        print("X shape: ", X_group.shape)
        print("Average Salary: ", np.mean(y_group.tolist()))
        print("Training...")
        test_score, train_score, model = train_model(model_selected, X_group, y_group)
        test_score_list.append(test_score)
        train_score_list.append(train_score)
        model_list.append(model)
        name_list.append(group)
        print("Training... done.")
                
    return test_score_list, train_score_list, model_list, name_list
    

In [19]:
test_score_list, train_score_list, model_list, name_list = estimate_salary(df_result, country_group_dict, country_list, country_group_list)

Cleaning the data...
Cleaning the data... done.
Trimming the data...
Trimming the data... done.
One-hot encoding...
One-hot encoding... done.
Trainning on: Without Country
Training... done.
Trainning on: With Country
Training... done.
Trainning on:  US
Filtering data...
X shape:  (12873, 124)
Average Salary:  118845.34785986172
Training...
Training... done.
Trainning on:  Group A
Filtering data...
X shape:  (8202, 124)
Average Salary:  77589.84284320897
Training...
Training... done.
Trainning on:  Group B
Filtering data...
X shape:  (6964, 124)
Average Salary:  63697.66255025847
Training...
Training... done.
Trainning on:  Group D
Filtering data...
X shape:  (8876, 124)
Average Salary:  25673.493803515095
Training...
Training... done.
Trainning on:  Group C
Filtering data...
X shape:  (6921, 124)
Average Salary:  43247.60092472186
Training...
Training... done.


In [20]:
#Print training and testing score
for i, name in enumerate(name_list):
    print("Country Group: ", name)
    print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score_list[i], test_score_list[i]))
    

Country Group:  Without Country
The rsquared on the training data was 0.28174604186850494.  The rsquared on the test data was 0.26521182579898284.
Country Group:  With Country
The rsquared on the training data was 0.5503605839018111.  The rsquared on the test data was 0.5421881911207236.
Country Group:  US
The rsquared on the training data was 0.2887898067321697.  The rsquared on the test data was 0.29838381179759665.
Country Group:  Group A
The rsquared on the training data was 0.32073269774020463.  The rsquared on the test data was 0.26262115528849683.
Country Group:  Group B
The rsquared on the training data was 0.2987618062287989.  The rsquared on the test data was 0.31822764031657347.
Country Group:  Group D
The rsquared on the training data was 0.20023977061170717.  The rsquared on the test data was 0.18561904369132587.
Country Group:  Group C
The rsquared on the training data was 0.24919140145958085.  The rsquared on the test data was 0.19167885712422084.
